TODO :
- add an `evaluate` method in my HRNet architecture definition

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import time
import argparse
import os
import rasterio
import sys
from pprint import pprint

In [2]:
script_folder = '/Users/bgl/Ensta/3a/pfe/hedgedex/hedgedexv3.0/scripts/'
sys.path.append(script_folder)

In [3]:
import model as m
from config import config
from config import MODEL_CONFIGS

/Users/bgl/Ensta/3a/pfe/hedgedex/ENV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pprint(MODEL_CONFIGS)

{'hrnet18': CfgNode({'FINAL_CONV_KERNEL': 1, 'STAGE1': CfgNode({'NUM_MODULES': 1, 'NUM_BRANCHES': 1, 'NUM_BLOCKS': [4], 'NUM_CHANNELS': [64], 'BLOCK': 'BOTTLENECK', 'FUSE_METHOD': 'SUM'}), 'STAGE2': CfgNode({'NUM_MODULES': 1, 'NUM_BRANCHES': 2, 'NUM_BLOCKS': [4, 4], 'NUM_CHANNELS': [18, 36], 'BLOCK': 'BASIC', 'FUSE_METHOD': 'SUM'}), 'STAGE3': CfgNode({'NUM_MODULES': 4, 'NUM_BRANCHES': 3, 'NUM_BLOCKS': [4, 4, 4], 'NUM_CHANNELS': [18, 36, 72], 'BLOCK': 'BASIC', 'FUSE_METHOD': 'SUM'}), 'STAGE4': CfgNode({'NUM_MODULES': 3, 'NUM_BRANCHES': 4, 'NUM_BLOCKS': [4, 4, 4, 4], 'NUM_CHANNELS': [18, 36, 72, 144], 'BLOCK': 'BASIC', 'FUSE_METHOD': 'SUM'})}),
 'hrnet32': CfgNode({'FINAL_CONV_KERNEL': 1, 'STAGE1': CfgNode({'NUM_MODULES': 1, 'NUM_BRANCHES': 1, 'NUM_BLOCKS': [4], 'NUM_CHANNELS': [64], 'BLOCK': 'BOTTLENECK', 'FUSE_METHOD': 'SUM'}), 'STAGE2': CfgNode({'NUM_MODULES': 1, 'NUM_BRANCHES': 2, 'NUM_BLOCKS': [4, 4], 'NUM_CHANNELS': [32, 64], 'BLOCK': 'BASIC', 'FUSE_METHOD': 'SUM'}), 'STAGE3': CfgN

In [6]:
model = m.get_seg_model(config)

KeyError: 'STAGE1'

In [7]:
config.defrost()
config.merge_from_file('/Users/bgl/Ensta/3a/pfe/hedgedex/hedgedexv3.0/test_config.yaml')
config.freeze()


In [8]:
model = m.get_seg_model(config)

In [9]:
model

HighResolutionNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): SyncBatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): 